In [1]:
import pandas as pd

#The data file Gungor_2018_VictorianAuthorAttribution_data-train.csv can be obtained from https://archive.ics.uci.edu/ml/machine-learning-databases/00454/
path_to_datafile = '..//..//DS7004//u1720146_DS7004_courseworkCodeAndData//preparationWorks//fromDS7003_Gungor2018VictorianAuthorAttribution_NGram//Gungor_2018_VictorianAuthorAttribution_data-train.csv'
pathToGungorVict = path_to_datafile
gungorVictRow = pd.read_csv(pathToGungorVict, encoding = 'ISO-8859-1')

In [2]:
#Use three authors' data: author:8 Charles Dickens total lines: 6914/ 14 George Eliot 2696/ 26 Jane Austen 4441
#each first 2500 lines for training, last 20 lines for testing. Each line has 1000 words
for i in [14, 26, 8]:
    allLines = gungorVictRow.loc[gungorVictRow['author'] == i]
    lines2500 = allLines.iloc[0:2500]
    linesLast20 = allLines.iloc[-20:]
    try:
        train = train.append(lines2500)
        test = test.append(linesLast20)
    except:
        train = lines2500
        test = linesLast20
train = train.sample(frac=1, random_state=42).reset_index(drop = True) #
test = test.sample(frac=1, random_state=42).reset_index(drop = True)

In [3]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def text_to_wordlist( text, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    text = BeautifulSoup(text).get_text()
    #  
    # 2. Remove non-letters
    text = re.sub("[^a-zA-Z]"," ", text)
    #
    # 3. Convert words to lower case and split them
    words = text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:  #These three lines will not be used. Pleasesee the second parameter of this function
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [4]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
#nltk.download() #no need to use this line again after it was used once  

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [5]:
# Define a function to split a text into parsed sentences
def text_to_sentences( text, tokenizer, remove_stopwords=False ):
    # Function to split a text into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(text.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call text_to_wordlist to get a list of words
            sentences.append( text_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [6]:
def parsing_sentence_set(text_df):
    sentences = []  # Initialize an empty list of sentences

    print("Parsing sentences from training set")
    for text in text_df["text"]:
        sentences += text_to_sentences(text, tokenizer)
    return sentences

In [7]:
sentences = parsing_sentence_set(train)

Parsing sentences from training set


In [8]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for the single neural network layer's various parameters
#num_features = 300    # Word vector dimensionality                      
#min_word_count = 40   # Minimum word count                        
#num_workers = 4       # Number of threads to run in parallel
#context = 10          # Context window size                                                                                    
#downsampling = 1e-3   # Downsample setting for frequent words

num_features = 400    # Word vector dimensionality                      
min_word_count = 5    # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 6           # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words
epochs= 25             #number of epochs

# Initialize and train the model (this will take some time)
# need to install gensim
from gensim.models import word2vec
def form_model_from_sentences(sentences):
    print("Training model...")
    model = word2vec.Word2Vec(sentences, workers=num_workers, \
                size=num_features, min_count = min_word_count, \
                window = context, sample = downsampling, iter = epochs)

    # If you don't plan to train the model any further, calling 
    # init_sims will make the model much more memory-efficient.
    model.init_sims(replace=True)
    return model

In [9]:
model = form_model_from_sentences(sentences)

2020-08-22 22:03:36,717 : INFO : collecting all words and their counts
2020-08-22 22:03:36,718 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model...


2020-08-22 22:03:37,887 : INFO : collected 9976 word types from a corpus of 7435641 raw words and 7500 sentences
2020-08-22 22:03:37,888 : INFO : Loading a fresh vocabulary
2020-08-22 22:03:37,903 : INFO : effective_min_count=5 retains 9940 unique words (99% of original 9976, drops 36)
2020-08-22 22:03:37,904 : INFO : effective_min_count=5 leaves 7435530 word corpus (99% of original 7435641, drops 111)
2020-08-22 22:03:37,931 : INFO : deleting the raw counts dictionary of 9976 items
2020-08-22 22:03:37,932 : INFO : sample=0.001 downsamples 55 most-common words
2020-08-22 22:03:37,932 : INFO : downsampling leaves estimated 5250158 word corpus (70.6% of prior 7435530)
2020-08-22 22:03:37,958 : INFO : estimated required memory for 9940 words and 400 dimensions: 36778000 bytes
2020-08-22 22:03:37,959 : INFO : resetting layer weights
2020-08-22 22:03:39,697 : INFO : training model with 4 workers on 9940 vocabulary and 400 features, using sg=0 hs=0 sample=0.001 negative=5 window=6
2020-08-22

2020-08-22 22:04:17,148 : INFO : EPOCH 8 - PROGRESS: at 68.93% examples, 716918 words/s, in_qsize 7, out_qsize 0
2020-08-22 22:04:18,149 : INFO : EPOCH 8 - PROGRESS: at 89.73% examples, 778988 words/s, in_qsize 7, out_qsize 0
2020-08-22 22:04:18,626 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-08-22 22:04:18,638 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-22 22:04:18,640 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-22 22:04:18,643 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-22 22:04:18,644 : INFO : EPOCH - 8 : training on 7435641 raw words (5249115 effective words) took 6.5s, 802433 effective words/s
2020-08-22 22:04:19,653 : INFO : EPOCH 9 - PROGRESS: at 19.47% examples, 1014957 words/s, in_qsize 7, out_qsize 0
2020-08-22 22:04:20,678 : INFO : EPOCH 9 - PROGRESS: at 40.53% examples, 1047197 words/s, in_qsize 7, out_qsize 0
2020-08-22 22:04:21,688 : INFO : EPOCH 9 - PR

2020-08-22 22:05:00,948 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-22 22:05:00,949 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-22 22:05:00,952 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-22 22:05:00,953 : INFO : EPOCH - 15 : training on 7435641 raw words (5250053 effective words) took 6.2s, 851838 effective words/s
2020-08-22 22:05:01,960 : INFO : EPOCH 16 - PROGRESS: at 19.60% examples, 1023773 words/s, in_qsize 7, out_qsize 0
2020-08-22 22:05:02,963 : INFO : EPOCH 16 - PROGRESS: at 41.47% examples, 1083665 words/s, in_qsize 7, out_qsize 0
2020-08-22 22:05:03,966 : INFO : EPOCH 16 - PROGRESS: at 54.13% examples, 943472 words/s, in_qsize 7, out_qsize 0
2020-08-22 22:05:04,973 : INFO : EPOCH 16 - PROGRESS: at 66.40% examples, 867143 words/s, in_qsize 7, out_qsize 0
2020-08-22 22:05:05,978 : INFO : EPOCH 16 - PROGRESS: at 79.33% examples, 829100 words/s, in_qsize 7, out_qsize 0
2020-08-22 22:05:

2020-08-22 22:05:43,629 : INFO : EPOCH - 22 : training on 7435641 raw words (5249574 effective words) took 5.9s, 892166 effective words/s
2020-08-22 22:05:44,641 : INFO : EPOCH 23 - PROGRESS: at 12.40% examples, 646672 words/s, in_qsize 7, out_qsize 0
2020-08-22 22:05:45,646 : INFO : EPOCH 23 - PROGRESS: at 30.80% examples, 804035 words/s, in_qsize 7, out_qsize 0
2020-08-22 22:05:46,655 : INFO : EPOCH 23 - PROGRESS: at 53.07% examples, 922213 words/s, in_qsize 7, out_qsize 0
2020-08-22 22:05:47,656 : INFO : EPOCH 23 - PROGRESS: at 73.20% examples, 955596 words/s, in_qsize 7, out_qsize 0
2020-08-22 22:05:48,658 : INFO : EPOCH 23 - PROGRESS: at 90.40% examples, 945007 words/s, in_qsize 7, out_qsize 0
2020-08-22 22:05:49,357 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-08-22 22:05:49,364 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-22 22:05:49,381 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-22 22:05:49

In [10]:
model.most_similar(positive=['king', 'woman'], negative=['man'])

<ipython-input-10-95bb2fc5eda1>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar(positive=['king', 'woman'], negative=['man'])


[('queen', 0.32510077953338623),
 ('conqueror', 0.30483680963516235),
 ('reign', 0.2798493504524231),
 ('saxon', 0.2771627902984619),
 ('throne', 0.2653134763240814),
 ('earl', 0.2596099376678467),
 ('child', 0.2358534038066864),
 ('girl', 0.23505660891532898),
 ('dying', 0.23019219934940338),
 ('kings', 0.22897280752658844)]

In [11]:
import gensim
all_stopwords = set(gensim.parsing.preprocessing.STOPWORDS)

In [12]:
#be careful: nword and counter must be integers --Chiu
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    index2word_set2 = all_stopwords
    #
    # Loop over each word in the text and, if it is in the model's
    # vocaublary and is a stop word add its feature vector to the total
    for word in words:
        if word in index2word_set and word in index2word_set2: 
            nwords = nwords + 1
            featureVec = np.add(featureVec, model[word])
    # 
    # Divide the result by the number of words to get the average
    if nwords == 0:
        nwords = 1 #avoid devided by zero (i.e. no stop word)
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(texts, model, num_features):
    # Given a set of texts (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    textFeatureVecs = np.zeros((len(texts),num_features),dtype="float32")
    # 
    # Loop through the texts
    for text in texts:
       #
       # Print a status message every 100th text
        if counter%100 == 0:
            haha = counter; hihi = len(texts)
            print(f"Text {haha} of {hihi}") #% (counter, len(texts))
       # 
       # Call the function (defined above) that makes average feature vectors
        #textFeatureVecs[counter] = makeFeatureVec(text, model, num_features)
        textFeatureVecs[counter] = makeFeatureVec(text, model, num_features)
       # Increment the counter
        counter = counter + 1
    return textFeatureVecs

In [35]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. 
num_features = 400
clean_train_texts = []
for text in train["text"]:
    #clean_train_reviews.append( review_to_wordlist( review, \
        #remove_stopwords=True )) #do not remove stop words
    clean_train_texts.append( text_to_wordlist( text ))

trainDataVecs = getAvgFeatureVecs( clean_train_texts, model, num_features )

print("Creating average feature vecs for test texts")
clean_test_texts = []
for text in test["text"]:
    #clean_test_texts.append( text_to_wordlist( review, remove_stopwords=True ))
    clean_test_texts.append( text_to_wordlist( text ))

testDataVecs = getAvgFeatureVecs( clean_test_texts, model, num_features )

Text 0 of 7500
Text 100 of 7500


<ipython-input-12-cf446d39f079>:23: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  featureVec = np.add(featureVec,model[word])


Text 200 of 7500
Text 300 of 7500
Text 400 of 7500
Text 500 of 7500
Text 600 of 7500
Text 700 of 7500
Text 800 of 7500
Text 900 of 7500
Text 1000 of 7500
Text 1100 of 7500
Text 1200 of 7500
Text 1300 of 7500
Text 1400 of 7500
Text 1500 of 7500
Text 1600 of 7500
Text 1700 of 7500
Text 1800 of 7500
Text 1900 of 7500
Text 2000 of 7500
Text 2100 of 7500
Text 2200 of 7500
Text 2300 of 7500
Text 2400 of 7500
Text 2500 of 7500
Text 2600 of 7500
Text 2700 of 7500
Text 2800 of 7500
Text 2900 of 7500
Text 3000 of 7500
Text 3100 of 7500
Text 3200 of 7500
Text 3300 of 7500
Text 3400 of 7500
Text 3500 of 7500
Text 3600 of 7500
Text 3700 of 7500
Text 3800 of 7500
Text 3900 of 7500
Text 4000 of 7500
Text 4100 of 7500
Text 4200 of 7500
Text 4300 of 7500
Text 4400 of 7500
Text 4500 of 7500
Text 4600 of 7500
Text 4700 of 7500
Text 4800 of 7500
Text 4900 of 7500
Text 5000 of 7500
Text 5100 of 7500
Text 5200 of 7500
Text 5300 of 7500
Text 5400 of 7500
Text 5500 of 7500
Text 5600 of 7500
Text 5700 of 7500


In [36]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs, train["author"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"true_author":test["author"], "pred_author":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


In [37]:
print(output)

    true_author  pred_author
0            14           14
1            14           14
2            26           26
3             8            8
4            14           14
5             8            8
6            26           26
7             8            8
8            14           14
9             8           14
10            8            8
11            8            8
12           26           26
13           14           14
14            8            8
15           14           14
16           14           14
17           14           14
18            8            8
19           14           14
20            8           14
21           14           14
22           26           26
23            8           14
24           26           26
25            8           14
26           14           14
27           26           26
28           14           14
29           26           26
30           26           26
31           14           14
32           26           26
33            

In [38]:
confusion_matrix = pd.crosstab(output['true_author'], output['pred_author'], rownames=['Actural'], colnames=['Predicted'])
print(confusion_matrix)

Predicted  8   14  26
Actural              
8          15   5   0
14          2  18   0
26          0   1  19


In [39]:
import sklearn
sklearn.metrics.confusion_matrix(output['true_author'], output['pred_author'])

array([[15,  5,  0],
       [ 2, 18,  0],
       [ 0,  1, 19]], dtype=int64)

In [40]:
sklearn.metrics.precision_recall_fscore_support(output['true_author'], output['pred_author'], average=None)

(array([0.88235294, 0.75      , 1.        ]),
 array([0.75, 0.9 , 0.95]),
 array([0.81081081, 0.81818182, 0.97435897]),
 array([20, 20, 20], dtype=int64))

In [41]:
pd.DataFrame(output.true_author.value_counts())

,true_author
26,20
14,20
8,20


In [42]:
word_vectors = model.wv.syn0

<ipython-input-42-98a1d4cbf9c6>:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  word_vectors = model.wv.syn0


In [43]:
word_vectors.shape

(9940, 400)

In [44]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.wv.syn0
num_clusters = word_vectors.shape[0] / 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = int(num_clusters) )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print("Time taken for K Means clustering: ", elapsed, "seconds.")

<ipython-input-44-c310dcc16c02>:8: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  word_vectors = model.wv.syn0


Time taken for K Means clustering:  637.9258322715759 seconds.


In [45]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.wv.index2word, idx ))

In [46]:
import operator

In [47]:
max(word_centroid_map, key=word_centroid_map.get)

'sins'

In [48]:
word_centroid_map.get('gentle')

559

In [49]:
word_centroid_map, len(word_centroid_map.keys())

({'the': 1577,
  'and': 1274,
  'to': 110,
  'of': 1620,
  'a': 360,
  'i': 92,
  'in': 1620,
  'was': 1252,
  'that': 776,
  'it': 776,
  'her': 503,
  'he': 74,
  'as': 1136,
  'she': 74,
  'you': 92,
  'with': 1274,
  'had': 338,
  'his': 105,
  'not': 1715,
  'for': 292,
  'be': 981,
  's': 667,
  'but': 755,
  'at': 138,
  'on': 335,
  'have': 484,
  'is': 1741,
  'my': 667,
  'by': 774,
  'said': 144,
  'him': 74,
  'me': 92,
  'which': 273,
  'mr': 1957,
  'all': 223,
  'so': 1006,
  'no': 1542,
  'this': 717,
  'been': 1029,
  'from': 334,
  'were': 252,
  'would': 110,
  'if': 1777,
  'an': 1067,
  'there': 1681,
  'what': 12,
  'when': 138,
  't': 92,
  'they': 463,
  'could': 110,
  'or': 1617,
  'very': 1006,
  'one': 788,
  'more': 284,
  'we': 92,
  'who': 641,
  'do': 92,
  'any': 1542,
  'mrs': 195,
  'will': 207,
  'are': 252,
  'them': 708,
  'little': 773,
  'out': 857,
  'than': 553,
  'their': 1798,
  'your': 667,
  'now': 92,
  'up': 1985,
  'some': 791,
  'much':

In [50]:
type(word_centroid_map)

dict

In [51]:
# For the first 10 clusters
for cluster in range(0,10):
    #
    # Print the cluster number  
    #print "\nCluster %d" #% cluster
    print(f"\nCluster {cluster}")
    #
    # Find all of the words for that cluster number, and print them out
    a_view = word_centroid_map.items()
    tuples = list(a_view)
    words = []
    for i in range(0,len(word_centroid_map.values())):
        if( tuples[i][1] == cluster ):
            words.append(tuples[i][0])
    print(words)


Cluster 0
['bred', 'educated']

Cluster 1
['spectacle', 'complicated', 'transaction', 'dealing', 'protracted', 'financial']

Cluster 2
['prove', 'serve', 'urge', 'cherish', 'identify']

Cluster 3
['frames', 'nightly', 'torch', 'crumbling']

Cluster 4
['hanging', 'bend', 'fastened', 'mantel', 'plucked', 'pinned']

Cluster 5
['leafy']

Cluster 6
['area', 'wards', 'halt']

Cluster 7
['safety', 'excess', 'suddenness']

Cluster 8
['land', 'bay', 'corn', 'lands', 'loth']

Cluster 9
['oil', 'ei', 'baron', 'fer', 'din']


In [52]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map and word in all_stopwords:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [53]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["text"].size, int(num_clusters)), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for text in clean_train_texts:
    train_centroids[counter] = create_bag_of_centroids( text, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros((test["text"].size, int(num_clusters)), \
    dtype="float32" )

counter = 0
for text in clean_test_texts:
    test_centroids[counter] = create_bag_of_centroids( text, \
        word_centroid_map )
    counter += 1

In [54]:
# This cell take some minutes
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train["author"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"true_author":test["author"], "pred_author":result})
output.to_csv( "BagOfCentroidsAuthor.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


In [55]:
output

,true_author,pred_author
0,14,14
1,14,14
2,26,26
3,8,8
4,14,14
5,8,8
6,26,26
7,8,8
8,14,14
9,8,8


In [56]:
confusion_matrix = pd.crosstab(output['true_author'], output['pred_author'], rownames=['Actural'], colnames=['Predicted'])
print(confusion_matrix)

Predicted  8   14  26
Actural              
8          18   2   0
14          1  18   1
26          0   0  20


In [57]:
import sklearn
sklearn.metrics.confusion_matrix(output['true_author'], output['pred_author'])

array([[18,  2,  0],
       [ 1, 18,  1],
       [ 0,  0, 20]], dtype=int64)

In [58]:
sklearn.metrics.precision_recall_fscore_support(output['true_author'], output['pred_author'], average=None)

(array([0.94736842, 0.9       , 0.95238095]),
 array([0.9, 0.9, 1. ]),
 array([0.92307692, 0.9       , 0.97560976]),
 array([20, 20, 20], dtype=int64))